In [1]:
# !conda install -c conda-forge ipympl

# If using JupyterLab
# !conda install -c conda-forge nodejs
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-matplotlib

In [2]:
import numpy as np
from utils.data_loading import read_csv
from modules.coincidence_counts import count_coincidences

In [3]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
%matplotlib widget

# Characterization of the single photon source

## Coincidence counts as a function of coincidence window width

In [4]:
sync, time = read_csv('measurements/coincidence_counts/window_width/5_2_1_window_width.csv')

In [5]:
# time includes the timestamp of each event
time[0:10]

array([1255892., 2579582., 2636980., 3260015., 3903854., 4019483.,
       4908083., 4987669., 6296538., 6653431.])

In [6]:
total_time = time[-1]
print('total measurement time: {} s'.format(total_time*1e-12))

total measurement time: 14.999998040321 s


In [7]:
# sync contains the channel id
sync[0:10]

array([1., 1., 2., 1., 2., 1., 2., 1., 2., 2.])

In [8]:
coinc_windows = np.arange(0, 5000, 100)
coinc_windows = np.append(coinc_windows, np.arange(5000, 85000, 5000))

coinc_rate = np.array([], dtype=np.float64)
accidental_coinc_rate = np.array([], dtype=np.float64)

for coinc_window in coinc_windows:
    coinc_rate = np.append(coinc_rate, count_coincidences(time, sync, coinc_window=coinc_window)/(total_time*1e-3))
    accidental_coinc_rate = np.append(accidental_coinc_rate, coinc_window*1e-3*np.sum(np.where(sync == 1, 1, 0))*np.sum(np.where(sync == 2, 1, 0))/(total_time*1e-3)**2)

In [9]:
fig, ax = plt.subplots()
ax.scatter(coinc_windows/1000, coinc_rate, marker='+')
ax.scatter(coinc_windows/1000, accidental_coinc_rate, marker='x')


ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
ax.set_xlabel('Coincidence window [ns]')
ax.set_ylabel('Coincidence rate [1/ns]') 

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Coincidence counts as a function of pump power

In [10]:
voltages = {0.2:"V0020.csv", 0.39:"V0390.csv", 0.8:"V0800.csv", 1.2:"V120.csv", 1.61:"V161.csv"}

coinc_rate = np.array([], dtype=np.float64)
accidental_coinc_rate = np.array([], dtype=np.float64)

coinc_window = 5000

for file in voltages.values():
    sync, time = read_csv('measurements/coincidence_counts/pump_power/5_2_1_'+file)
    coinc_rate = np.append(coinc_rate, count_coincidences(time, sync, coinc_window=coinc_window)/(total_time*1e-3))
    accidental_coinc_rate = np.append(accidental_coinc_rate, coinc_window*1e-3*np.sum(np.where(sync == 1, 1, 0))*np.sum(np.where(sync == 2, 1, 0))/(total_time*1e-3)**2)    

In [11]:
fig, ax = plt.subplots()
color = 'tab:red'
ax.scatter(voltages.keys(), coinc_rate, marker='+', color=color)
ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
ax.tick_params(axis='y', labelcolor=color)
ax.set_xlabel('Spannung [V]')
ax.set_ylabel('Koinzidenzrate [1/ns]') 

color = 'tab:blue'
ax_2 = ax.twinx()
ax_2.set_ylabel('Zufällige Koinzidenzrate [1/ns]')
ax_2.scatter(voltages.keys(), accidental_coinc_rate, marker='x', color=color)
ax_2.tick_params(axis='y', labelcolor=color)
fig.tight_layout()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Coincidence counts as a function of pump polarization

In [15]:
import os

data_dir = 'measurements/coincidence_counts/pump_polarization'

polarizations = np.array([])
coinc_rate = np.array([], dtype=np.float64)

coinc_window = 5000

for file_name in os.listdir(data_dir):
    
    base_name, ext = os.path.splitext(file_name)
    if ext != '.csv':
        continue
    
    # we named each file d<degrees>.csv so we can just extract the value
    deg = int(base_name.split('d')[-1])
    
    polarizations = np.append(polarizations, deg)
    
    sync, time = read_csv(os.path.join(data_dir, file_name))
    coinc_rate = np.append(coinc_rate, count_coincidences(time, sync, coinc_window=coinc_window)/(total_time*1e-3))

In [20]:
fig, ax = plt.subplots()
ax.plot(polarizations, coinc_rate, linestyle='--', marker='o')

ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
ax.set_xlabel('Rotation angle')
ax.set_ylabel('Coincidence rate [1/ns]') 

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Measurement of the first order correlation function

## Single Photon Heralded

In [42]:
import tqdm

data_dir = 'measurements/g1_correlation_function/single_photon/'

voltages = np.array([])
coinc_rate = np.array([], dtype=np.float64)
coinc_rate_unheralded = np.array([], dtype=np.float64)

coinc_window = 5000

for file_name in tqdm.tqdm(os.listdir(data_dir)):
    
    base_name, ext = os.path.splitext(file_name)
    if ext != '.csv':
        continue
    
    # load voltage g1_XX_XX
    voltages = np.append(voltages, float(base_name.split('_')[1]) + float(base_name.split('_')[2])/100)
    
    sync, time = read_csv(os.path.join(data_dir, file_name))
    coinc_rate = np.append(coinc_rate, count_coincidences(time, sync, coinc_window=coinc_window)/(total_time*1e-3))
    
    coinc_rate_unheralded = np.append(coinc_rate_unheralded, np.sum(np.where(sync == 1, 1, 0))/(total_time*1e-3))

100%|██████████| 260/260 [06:27<00:00,  1.49s/it]


In [43]:
fig, ax = plt.subplots()
ax.plot(voltages, coinc_rate)

ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
ax.set_xlabel('Voltage [V]')
ax.set_ylabel('Coincidence rate [1/ns]')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Single Photon Heralded

In [44]:
fig, ax = plt.subplots()
ax.plot(voltages, coinc_rate_unheralded)

ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
ax.set_xlabel('Voltage [V]')
ax.set_ylabel('Coincidence rate [1/ns]') 

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Coherent State

In [45]:
data_dir = 'measurements/g1_correlation_function/coherent/'

voltages = np.array([])
coinc_rate = np.array([], dtype=np.float64)

coinc_window = 5000

for file_name in tqdm.tqdm(os.listdir(data_dir)):
    
    base_name, ext = os.path.splitext(file_name)
    if ext != '.csv':
        continue
    
    # load voltage g1_XX_XX
    voltages = np.append(voltages, float(base_name.split('_')[1]) + float(base_name.split('_')[2])/100)
    
    sync, time = read_csv(os.path.join(data_dir, file_name))
    coinc_rate = np.append(coinc_rate, np.sum(np.where(sync == 1, 1, 0))/(total_time*1e-3))

100%|██████████| 172/172 [03:35<00:00,  1.25s/it]


In [51]:
fig, ax = plt.subplots()
ax.plot(voltages, coinc_rate)

ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
ax.set_xlabel('Voltage [V]')
ax.set_ylabel('Coincidence rate [1/ns]') 

plt.show()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …